<h1> Polymoprhic Eptiope Prediction </h1>

This tutorial illustrates how to use Fred2 to integrate genomic information of a patient for epitope prediction

This tutorial will entail:
- Variant construction
- Polymorphic Transcript/Protein/Peptide construction
- Polymorphic epitope prediction


In [21]:
%matplotlib inline
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

#is is needed as long as Fred2 is not installed properly
import sys 

sys.path.append("/Users/schubert/Dropbox/PhD/Porgramming/Fred2")
from Fred2.Core import Allele, Peptide, Protein,generate_peptides_from_protein
from Fred2.IO import read_lines, read_fasta
from Fred2.EpitopePrediction import EpitopePredictorFactory
from Fred2.Core import generate_transcripts_from_variants
from Fred2.IO import read_annovar_exonic
from Fred2.IO.MartsAdapter import MartsAdapter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
#id, type, line, chrom, genome_start, genome_stop, ref, alt, zygos
vars = read_annovar_exonic("./data/test_annovar.out")
print vars

trans = list(generate_transcripts_from_variants(vars, MartsAdapter()))
print trans

[Variant(g.67705958G>A), Variant(g.234183368A>G), Variant(g.50745926C>T), Variant(g.50756540G>C), Variant(g.20763686G>-)]
NM_004004
NM_001293557
NM_001190267
NM_144701
NM_001190266

/Users/schubert/Dropbox/PhD/Porgramming/Fred2/Fred2/Core/Generator.py:51: UserWarning: For NM_001293557 bp does not match ref of assigned variant Variant(g.50756540G>C). Pos 2640, var ref G, seq ref A 
  seq[pos]))
/Users/schubert/Dropbox/PhD/Porgramming/Fred2/Fred2/Core/Generator.py:51: UserWarning: For NM_001293557 bp does not match ref of assigned variant Variant(g.50745926C>T). Pos 2022, var ref C, seq ref G 
  seq[pos]))
/Users/schubert/Dropbox/PhD/Porgramming/Fred2/Fred2/Core/Generator.py:51: UserWarning: For NM_001190266 bp does not match ref of assigned variant Variant(g.234183368A>G). Pos 645, var ref A, seq ref C 
  seq[pos]))



NM_198890
NM_022162
NM_030803
NM_017974
[TRANSCRIPT: NM_004004:FRED2_0
	VARIANTS:
	SEQUENCE: ATGGATTGGGGCACGCTGCAGACGATCCTGGGGGGTGTGAACAAACACTCCACCAGCATTGGAAAGATCTGGCTCACCGTCCTCTTCATTTTTCGCATTATGATCCTCGTTGTGGCTGCAAAGGAGGTGTGGGGAGATGAGCAGGCCGACTTTGTCTGCAACACCCTGCAGCCAGGCTGCAAGAACGTGTGCTACGATCACTACTTCCCCATCTCCCACATCCGGCTATGGGCCCTGCAGCTGATCTTCGTGTCCACGCCAGCGCTCCTAGTGGCCATGCACGTGGCCTACCGGAGACATGAGAAGAAGAGGAAGTTCATCAAGGGGGAGATAAAGAGTGAATTTAAGGACATCGAGGAGATCAAAACCCAGAAGGTCCGCATCGAAGGCTCCCTGTGGTGGACCTACACAAGCAGCATCTTCTTCCGGGTCATCTTCGAAGCCGCCTTCATGTACGTCTTCTATGTCATGTACGACGGCTTCTCCATGCAGCGGCTGGTGAAGTGCAACGCCTGGCCTTGTCCCAACACTGTGGACTGCTTTGTGTCCCGGCCCACGGAGAAGACTGTCTTCACAGTGTTCATGATTGCAGTGTCTGGAATTTGCATCCTGCTGAATGTCACTGAATTGTGTTATTTGCTAATTAGATATTGTTCTGGGAAGTCAAAAAAGCCAGTTTAA (mRNA), TRANSCRIPT: NM_004004:FRED2_1
	VARIANTS:
		pos 34: Variant(g.20763686C>-)
	SEQUENCE: ATGGATTGGGGCACGCTGCAGACGATCCTGGGGGTGTGAACAAACACTCCACCAGCATTGGAAAGATCTGGCTCACCGTCCTCTTCATTTTTCGCATTATGATCCTCGTTGTGGCTGCAAAGGAGGTGTGG